# 1. Bag-of-Words (단어 가방 만들기)

input()	: 키보드 입력 및 standard input 을 받아들입니다.
<br>.lower() 	: 주어진 문자열을 소문자로 치환합니다.
<br>.split()	: 주어진 문자열을 () 내의 기준으로 잘라 list 형식( [x, y, …, z] ) 저장하여 반환합니다.
<br>.len()	: 주어진 문자열의 길이를 반환합니다.


re.sub(r'[^a-z]+', ' ', sentence)	: re는 정규표현식과 관련된 파이썬 내부 패키지입니다.

- sub(pattern, repl, string[, count=0]) 
<br>-> string에서 pattern과 일치하는 부분에 대하여 repl로 교체하여 결과 문자열을 반환하는 함수입니다.

- r'[^a-z]+' 에서 r'~' 은 정규표현식을 다룬다는 것을 뜻합니다.
- [^~] == ~가 아닌 것 (a~z 이 아닌 것)
- 즉, 알파벳 소문자가 아닌 것을 한 글자 혹은 여러 글자 단위(+)로 찾아 space(공백)으로 치환합니다.


In [1]:
import re

def main():
    sentence = input() # : 키보드 입력 및 standard input 을 받아들입니다. 
    BOW = create_BOW(sentence)

    print(BOW)

def create_BOW(sentence):
    bow = {}
    
    sentence = sentence.lower() # : 주어진 문자열을 소문자로 치환합니다. 
    sentence = replace_non_alphabetic_chars_to_space(sentence)
    sentence = sentence.split(" ") # :주어진 문자열을 () 내의 기준으로 잘라 list 형식( [x, y, …, z] ) 저장하여 반환합니다.
    
    for word in sentence:
        if len(word) >= 1:
            if word in bow:
                bow[word] += 1
            else:
                bow[word] = 1
    return bow

# 알파벳 소문자가 아닌 것을 한 글자 혹은 여러 글자 단위(+)로 찾아 space(공백)으로 치환합니다.
def replace_non_alphabetic_chars_to_space(sentence):
    return re.sub(r'[^a-z]+', ' ', sentence)


if __name__ == "__main__":
    main()

John likes to watch movies. Mary likes movies too. Let's go!
{'let': 1, 'too': 1, 's': 1, 'movies': 2, 'to': 1, 'watch': 1, 'go': 1, 'likes': 2, 'john': 1, 'mary': 1}


# 2. Likelihood & Laplace Smoothing 실습

본 실습에서는 함수 calculate_doc_prob 에서 Laplace smoothing을 이용해, 
<br>트레이닝 데이터에서 생성된 bag-of-words 모델이 테스트 데이터를 생성할 로그 확률 을 구합니다. 

로그 확률을 구하는 이유는, 
<br>긴 문장의 경우 생성 확률값이 매우 작아져 Python 에서 기본으로 사용하는 floating point 형식에서 오차가 발생하기 때문입니다.

In [3]:
import re
import math

def main():
    # input() =
    # John likes to watch movies. Mary likes movies too.
    # John also likes to watch football games.
    # 0.1
        
    # 1
    training_sentence = input()
    training_model = create_BOW(training_sentence)

    # 2
    testing_sentence = input()
    testing_model = create_BOW(testing_sentence)

    # 3
    alpha = float(input())

    print(calculate_doc_prob(training_model, testing_model, alpha))

    
def calculate_doc_prob(training_model, testing_model, alpha):
    
    logprob = 0   # (필수) log probability 값 초기화 
    N = sum(training_model.values())  # 트레이닝 데이터 내 전체 단어 갯수
    d = len(training_model.keys())  # == len(training_model), 트레이닝 데이터 내 서로 다른 단어 갯수

    for word in testing_model:
        
        if word in training_model:
            count = training_model[word]
        else: 
            count = 0
            
        # 자연로그 함수(math.log()) 대신 직관적인 상용로그 함수를 사용했습니다. 
        logprob += math.log10((count + alpha) / (N + alpha * d)) * testing_model[word]
        # ex) "also"가 2차례 등장 시, [ * testing_model["also"] = * 2 ] 
        # -> "also" 에 해당하는 log를 2번 더한 것이므로, log 내부로 들어가서는 곱셈으로 처리됨 
        # ->> "also" 에 해당하는 확률에 ^2 를 해준 것과 같은 결과 
        # ->>> 동전 예시의 [ L(θ1|x)=P(x|θ) = 0.7^8 * (1-0.7)^2 ~ 0.005188 ] 에서 ^8 or ^2 와 같은 처리를 해준 것 

    return logprob


def create_BOW(sentence):
    bow = {}
    
    sentence = sentence.lower()
    sentence = replace_non_alphabetic_chars_to_space(sentence)
    sentence = sentence.split(" ")  
    
    for word in sentence:
        if word in bow:
            bow[word] += 1  # word key 의 value 를 1 추가
        else:
            if len(word) != 0:  # 길이가 0인 "" 제거 
                bow[word] = 1  # word key 의 value == 1

    return bow


def replace_non_alphabetic_chars_to_space(sentence):
    return re.sub(r'[^a-z]+', ' ', sentence)


if __name__ == "__main__":
    main()


John likes to watch movies. Mary likes movies too.
John also likes to watch football games.
0.1
-9.461004789655119


# 3. Naive Bayes Classifier 구현하기


여러 개의 이미 트레이닝된 모델이 있을 때, 
<br>주어진 텍스트가 어떤 모델에 더 적합한지 판정 (classify) 하는 방법론을 코딩을 통해 구현해보겠습니다. 

In [4]:
import re
import math
import naivebayes_utils

def main():
    training1_sentence = input()
    training2_sentence = input()
    testing_sentence = input()

    alpha = float(input())
    prob1 = float(input())
    prob2 = float(input())

    print(naive_bayes(training1_sentence, training2_sentence, testing_sentence, alpha, prob1, prob2))

    
def naive_bayes(training1_sentence, training2_sentence, testing_sentence, alpha, prob1, prob2):
    
    bow_train1 = create_BOW(training1_sentence)
    bow_train2 = create_BOW(training2_sentence)
    bow_test = create_BOW(testing_sentence)
    
       
    # 0. p( C1 | x ) ∝ p( C1 ) * p( x | C1 ) 
    # 1.1. p( C1 | x ) 자체가 로그값이 취해진 확률값이 되어야 하므로,
    # 1.2. log { p( C1 ) * p( x | C1 ) } = log { p(C1) } + log { p( x | C1 ) }
    # 2.1. log { p(C1) } == math.log(prob1)
    # 2.2. log { p( x | C1 ) } == log_likelihood(bow_train1, bow_test, alpha)

    classify1 = math.log(prob1) + log_likelihood(bow_train1, bow_test, alpha)
    classify2 = math.log(prob2) + log_likelihood(bow_train2, bow_test, alpha)

    return normalize_log_prob(classify1, classify2)


def normalize_log_prob(prob1, prob2):
    return naivebayes_utils.normalize_log_prob(prob1, prob2)

def log_likelihood(training_model, testing_model, alpha):
    return naivebayes_utils.calculate_doc_prob(training_model, testing_model, alpha)

def create_BOW(sentence):
    return naivebayes_utils.create_BOW(sentence)

if __name__ == "__main__":
    main()

John likes to watch movies. Mary likes movies too.
In the machine learning, naive Bayes classifiers are a family of simple probabilistic classifiers.
John also likes to watch football games.
0.1
0.5
0.5
(0.9999985271309568, 1.4728690432649637e-06)


# 4. Naive Bayes로 감정 분석하기

현재까지 감정을 컴퓨터로 분석하려는 시도는 계속 진행중입니다. 
>크게 긍정적인 감정/부정적인 감정을 나누는 문제를 **sentiment analysis (정서 분석)** 이라고 합니다. 
<br>이 외에 사람들이 느끼는 감정들, 
<br>예를 들어 분노/기쁨/놀람/사랑/슬픔/두려움 (Parrott’s Emotions) 혹은 
<br>기쁨/신뢰/두려움/놀람/슬픔/역겨움/화남/기대 (Plutchik’s Wheel of Emotions) 등의 분석을 하는 문제를 <br>**emotion analysis (감정 분석)** 이라고 합니다.

실제 리뷰를 데이터를 통해 Naive Bayes Classifier를 트레이닝하고, 
<br>입력받을 문장이 positive일 확률 및 negative일 확률을 구하는 문제 (sentiment analysis) 를 실습하겠습니다.


In [ ]:
import io
import numpy
import naivebayes_utils
import re
import math
import os

def main():
    training1_sentences = read_text_data('./txt_sentoken/pos/')
    training2_sentences = read_text_data('./txt_sentoken/neg/')
    testing_sentence = input()

    alpha = 0.1
    prob1 = 0.5
    prob2 = 0.5

    prob_pair = naive_bayes(training1_sentences, training2_sentences, testing_sentence, alpha, prob1, prob2)

    print(prob_pair)
    
    # 그래프로 긍정/부정 결과를 출력 1 (테스트 문장이 길 경우, 말줄임표 처리)
#     plot_title = testing_sentence
#     if len(plot_title) > 50: plot_title = plot_title[:50] + "..."
    
    # 그래프로 긍정/부정 결과를 출력 2 (elice 의 패키지를 활용하여 확률값을 그림으로 표현)
#     visualize_boxplot(plot_title,
#                   list(prob_pair),
#                   ['Positive', 'Negative'])


def naive_bayes(training1_sentence, training2_sentence, testing_sentence, alpha, prob1, prob2):
    
    # 먼저 전체 텍스트 문장으로 각각에 대하여 BOW 를 만들어줘야 한다.
    bow_train1 = create_BOW(training1_sentence)
    bow_train2 = create_BOW(training2_sentence)
    bow_test = create_BOW(testing_sentence)
    
    # 0. p( C1 | x ) ∝ p( C1 ) * p( x | C1 ) 
    # 1.1. p( C1 | x ) 자체가 로그값이 취해진 확률값이 되어야 하므로,
    # 1.2. log { p( C1 ) * p( x | C1 ) } = log { p(C1) } + log { p( x | C1 ) }
    # 2.1. log { p(C1) } == math.log(prob1)
    # 2.2. log { p( x | C1 ) } == log_likelihood(bow_train1, bow_test, alpha)

    classify1 = math.log(prob1) + calculate_doc_prob(bow_train1, bow_test, alpha)
    classify2 = math.log(prob2) + calculate_doc_prob(bow_train2, bow_test, alpha)

    return normalize_log_prob(classify1, classify2)


# 폴더 경로를 인자로 받아, 해당 폴더 내의 모든 txt 파일들의 모든 line 들을 읽어들여 all_text 에 저장하는 함수
def read_text_data(directory):
    files = os.listdir(directory)
    files = [f for f in files if f.endswith('.txt')]    # pos or neg 폴더 내의 .txt 파일들을 모두 읽어들임

    all_text = ''
    for f in files:
        all_text += ' '.join(open(directory + f).readlines()) + ' '

    return all_text


def normalize_log_prob(prob1, prob2):
    return naivebayes_utils.normalize_log_prob(prob1, prob2)

def calculate_doc_prob(training_model, testing_model, alpha):
    return naivebayes_utils.calculate_doc_prob(training_model, testing_model, alpha)

def create_BOW(sentence):
    return naivebayes_utils.create_BOW(sentence)

def visualize_boxplot(title, values, labels):
    naivebayes_utils.visualize_boxplot(title, values, labels)

if __name__ == "__main__":
    main()